In [9]:
import pandas as pd
import sqlite3
import re

connection = sqlite3.connect('../german_credit.db')

def select(query):
    try:
        pd.read_sql_query(query, connection)
    except:
        cursor.executescript(query)
        connection.commit()
    else:
        return pd.read_sql_query(query, connection)

def get_one_case_pivot(counting_var, grouping_var, value):
    return f"ROUND(AVG(CASE WHEN {grouping_var} = '{value}' THEN {counting_var} ELSE NULL END), 2) AS {re.sub(r'[\s/]', '_', value.lower())}"
    
def get_values_for_pivot(db_name, grouping_var):
    return select(fr"""SELECT DISTINCT {grouping_var} FROM {db_name}""")[f'{grouping_var}'].tolist()
    
def get_pivot_string(db_name, counting_var, grouping_var):
    return ',\n'.join([get_one_case_pivot(counting_var, grouping_var, x) for x in get_values_for_pivot(db_name, grouping_var)])
    
def get_pivot_avg(db_name, col_name, row_name, count_var):
    query = fr'''
    SELECT
        {row_name},
        {get_pivot_string(db_name, count_var, col_name)}
    FROM {db_name}
    GROUP BY {row_name}
    '''
    return select(query)

get_pivot_avg('german_credit', 'sex', 'job', 'duration')

,job,male,female
0,0,18.10,16.75
1,1,16.02,17.63
2,2,22.41,19.22
3,3,25.35,24.62


In [6]:
query = r'''
    SELECT *  
    FROM german_credit AS gc
    LIMIT 10
'''
select(query)

,age,sex,job,housing,saving_accounts,checking_account,credit_amount,duration,purpose,default,contract_dt,client_id
0,33,male,2,own,None,no_info,3074,9,radio/TV,0,2008-06-29 18:52:00,210
1,43,male,1,own,little,little,1344,12,car,0,2007-05-20 18:30:19,929
2,52,male,2,own,quite rich,no_info,936,9,education,0,2008-04-27 08:23:07,200
3,35,female,3,own,little,no_info,1393,11,car,0,2007-05-06 10:58:22,45
4,28,male,2,own,little,no_info,776,12,radio/TV,0,2007-07-21 13:22:14,358
5,27,female,2,own,little,moderate,1295,18,furniture/equipment,0,2008-06-18 04:10:05,86
6,29,female,2,own,little,moderate,959,9,furniture/equipment,1,2007-05-21 14:32:00,351
7,42,female,2,rent,rich,rich,409,12,radio/TV,0,2008-06-05 05:31:13,27
8,26,male,2,own,little,little,4370,42,radio/TV,1,2007-11-29 00:20:44,639
9,55,male,2,free,rich,no_info,1555,12,repairs,1,2007-09-11 20:27:28,754
